<h1>ENB32 - Revisiting FFNs with K-fold cross validation</h1>
<h2> dropout layer </h2>

In [1]:
import tensorflow as tf
import keras
from keras import models
from keras import layers

import numpy as np
import os
import csv

import pickle
import time as tm

import matplotlib.pyplot as plt
import pre_process as pp

Using TensorFlow backend.


<h2> Loading the datasets</h2>

In [2]:
with open('train_data.txt', 'rb') as fp:
    train_data = pickle.load(fp)
    
with open('train_labels.txt', 'rb') as fp:
    train_labels = pickle.load(fp)
    
with open('test_data.txt', 'rb') as fp:
    test_data = pickle.load(fp)
    
with open('test_labels.txt', 'rb') as fp:
    test_labels = pickle.load(fp)

<h2>k-fold params</h2>

In [3]:
k = 4
num_val_samples = len(train_data)
print(num_val_samples)
val_split_size = num_val_samples/k
val_split_size = int(val_split_size)
print(val_split_size)
type(val_split_size)

72
18


int

<h2>Experiment1: a single layer</h2>

In [4]:
net1 = models.Sequential()
net1.add(layers.Dense((480), input_shape = (480,6)))
net1.add(layers.Flatten())
net1.add(layers.Dropout(0.05))
net1.add(layers.Dense(240,activation='relu'))
net1.add(layers.Dense(240,activation='relu'))
net1.add(layers.Dense(240,activation='relu'))
net1.add(layers.Dense(7, activation = 'softmax'))
net1.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 480, 480)          3360      
_________________________________________________________________
flatten_1 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 230400)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 240)               55296240  
_________________________________________________________________
dense_3 (Dense)              (None, 240)               57840     
_________________________________________________________________
dense_4 (Dense)              (None, 240)               57840     
_______________________________________

In [5]:
net1.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net1.save_weights('net1.h5')  # each split will start with same weights so can directly compare results

In [6]:

histories1 = []
results_list1 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net1.load_weights('net1.h5')
    history = net1.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories1.append(history)
    
    
    results = net1.evaluate(test_data, test_labels)
    results_list1.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size
    
finish = tm.time()
duration1 = finish - start

0

Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 30ms/step - loss: 3.4223 - accuracy: 0.0926 - val_loss: 6.9496 - val_accuracy: 0.2778
Epoch 2/25
54/54 [==============================] - 2s 28ms/step - loss: 5.5795 - accuracy: 0.3333 - val_loss: 10.0709 - val_accuracy: 0.1667
Epoch 3/25
54/54 [==============================] - 2s 28ms/step - loss: 6.0854 - accuracy: 0.2963 - val_loss: 6.4975 - val_accuracy: 0.2222
Epoch 4/25
54/54 [==============================] - 2s 28ms/step - loss: 4.0438 - accuracy: 0.3704 - val_loss: 2.2322 - val_accuracy: 0.2222
Epoch 5/25
54/54 [==============================] - 2s 28ms/step - loss: 2.4240 - accuracy: 0.2037 - val_loss: 1.7438 - val_accuracy: 0.2778
Epoch 6/25
54/54 [==============================] - 2s 28ms/step - loss: 1.9822 - accuracy: 0.3148 - val_loss: 2.2805 - val_accuracy: 0.4444
Epoch 7/25
54/54 [==============================] - 2s 28ms/step - loss: 1.5477 - accuracy: 0.6296 - val_l

Epoch 5/25
54/54 [==============================] - 2s 28ms/step - loss: 2.5808 - accuracy: 0.1296 - val_loss: 2.4145 - val_accuracy: 0.2778
Epoch 6/25
54/54 [==============================] - 2s 28ms/step - loss: 1.9764 - accuracy: 0.3148 - val_loss: 2.2664 - val_accuracy: 0.2222
Epoch 7/25
54/54 [==============================] - 2s 28ms/step - loss: 1.6097 - accuracy: 0.3889 - val_loss: 1.8681 - val_accuracy: 0.3889
Epoch 8/25
54/54 [==============================] - 1s 28ms/step - loss: 1.5058 - accuracy: 0.4259 - val_loss: 1.5259 - val_accuracy: 0.3333
Epoch 9/25
54/54 [==============================] - 2s 28ms/step - loss: 1.2951 - accuracy: 0.4444 - val_loss: 1.0851 - val_accuracy: 0.5556
Epoch 10/25
54/54 [==============================] - 2s 28ms/step - loss: 0.9545 - accuracy: 0.6481 - val_loss: 1.2445 - val_accuracy: 0.5556
Epoch 11/25
54/54 [==============================] - 1s 28ms/step - loss: 0.8871 - accuracy: 0.7037 - val_loss: 1.2179 - val_accuracy: 0.5556
Epoch 12/25

In [7]:
avg = np.mean(results_list1, axis=0)
std_dev = np.std(results_list1, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration1))

Average loss and accuracy scores were:  [0.53015116 0.81666668]
Standard deviation for loss & accuracy:  [0.1028132  0.05527708]
Duration in seconds:  154


<h2> Expriment2:</h2>

In [8]:
net2 = models.Sequential()
net2.add(layers.Dense((480), input_shape = (480,6)))
net2.add(layers.Flatten())
net2.add(layers.Dropout(0.1))
net2.add(layers.Dense(240,activation='relu'))
net2.add(layers.Dense(240,activation='relu'))
net2.add(layers.Dense(240,activation='relu'))
net2.add(layers.Dense(7, activation = 'softmax'))
net2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 480, 480)          3360      
_________________________________________________________________
flatten_2 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 230400)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 240)               55296240  
_________________________________________________________________
dense_8 (Dense)              (None, 240)               57840     
_________________________________________________________________
dense_9 (Dense)              (None, 240)               57840     
_________________________________________________________________
dense_10 (Dense)             (None, 7)                

In [9]:
net2.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net2.save_weights('net2.h5')  # each split will start with same weights so can directly compare results

histories2 = []
results_list2 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net2.load_weights('net2.h5')
    history = net2.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories2.append(history)
    
    
    results = net2.evaluate(test_data, test_labels)
    results_list2.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration2 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 31ms/step - loss: 7.1576 - accuracy: 0.0741 - val_loss: 4.5281 - val_accuracy: 0.0556
Epoch 2/25
54/54 [==============================] - 2s 28ms/step - loss: 3.5207 - accuracy: 0.1667 - val_loss: 4.3471 - val_accuracy: 0.2778
Epoch 3/25
54/54 [==============================] - 2s 28ms/step - loss: 4.8400 - accuracy: 0.2222 - val_loss: 4.0229 - val_accuracy: 0.2778
Epoch 4/25
54/54 [==============================] - 2s 28ms/step - loss: 3.0452 - accuracy: 0.3889 - val_loss: 3.5197 - val_accuracy: 0.3889
Epoch 5/25
54/54 [==============================] - 1s 28ms/step - loss: 2.8315 - accuracy: 0.2963 - val_loss: 1.6853 - val_accuracy: 0.2222
Epoch 6/25
54/54 [==============================] - 2s 28ms/step - loss: 1.5416 - accuracy: 0.2778 - val_loss: 1.8227 - val_accuracy: 0.1111
Epoch 7/25
54/54 [==============================] - 2s 29ms/step - loss: 1.4442 - accuracy: 0.4259 - val_los

In [10]:
avg = np.mean(results_list2, axis=0)
std_dev = np.std(results_list2, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration2))

Average loss and accuracy scores were:  [0.48679763 0.80000001]
Standard deviation for loss & accuracy:  [0.06361265 0.04714045]
Duration in seconds:  154


<h2> Expriment3:</h2>

In [11]:
net3 = models.Sequential()
net3.add(layers.Dense((480), input_shape = (480,6)))
net3.add(layers.Flatten())
net3.add(layers.Dropout(0.15))
net3.add(layers.Dense(240,activation='relu'))
net3.add(layers.Dense(240,activation='relu'))
net3.add(layers.Dense(240,activation='relu'))
net3.add(layers.Dense(7, activation = 'softmax'))
net3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 480, 480)          3360      
_________________________________________________________________
flatten_3 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 230400)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 240)               55296240  
_________________________________________________________________
dense_13 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_14 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_15 (Dense)             (None, 7)                

In [12]:
net3.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net3.save_weights('net3.h5')  # each split will start with same weights so can directly compare results

histories3 = []
results_list3 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net3.load_weights('net3.h5')
    history = net3.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories3.append(history)
    
    
    results = net3.evaluate(test_data, test_labels)
    results_list3.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration3 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 31ms/step - loss: 5.3346 - accuracy: 0.1852 - val_loss: 8.4233 - val_accuracy: 0.1667
Epoch 2/25
54/54 [==============================] - 2s 28ms/step - loss: 7.9516 - accuracy: 0.1296 - val_loss: 7.4575 - val_accuracy: 0.1111
Epoch 3/25
54/54 [==============================] - 1s 28ms/step - loss: 5.5967 - accuracy: 0.1296 - val_loss: 3.7110 - val_accuracy: 0.5000
Epoch 4/25
54/54 [==============================] - 2s 29ms/step - loss: 3.8478 - accuracy: 0.2222 - val_loss: 2.4848 - val_accuracy: 0.1111
Epoch 5/25
54/54 [==============================] - 2s 29ms/step - loss: 2.4522 - accuracy: 0.2407 - val_loss: 2.2804 - val_accuracy: 0.3333
Epoch 6/25
54/54 [==============================] - 2s 28ms/step - loss: 2.3903 - accuracy: 0.3148 - val_loss: 2.0609 - val_accuracy: 0.3889
Epoch 7/25
54/54 [==============================] - 2s 28ms/step - loss: 1.9539 - accuracy: 0.3889 - val_los

In [14]:
avg = np.mean(results_list3, axis=0)
std_dev = np.std(results_list3, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration3))

Average loss and accuracy scores were:  [0.50958291 0.81666668]
Standard deviation for loss & accuracy:  [0.07518471 0.02886751]
Duration in seconds:  154


<h2> Expriment4:</h2>

In [21]:
net4 = models.Sequential()
net4.add(layers.Dense((480), input_shape = (480,6)))
net4.add(layers.Flatten())
net4.add(layers.Dropout(0.2))
net4.add(layers.Dense(240,activation='relu'))
net4.add(layers.Dense(240,activation='relu'))
net4.add(layers.Dense(240,activation='relu'))
net4.add(layers.Dense(7, activation = 'softmax'))
net4.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 480, 480)          3360      
_________________________________________________________________
flatten_6 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 230400)            0         
_________________________________________________________________
dense_27 (Dense)             (None, 240)               55296240  
_________________________________________________________________
dense_28 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_29 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_30 (Dense)             (None, 7)                

In [22]:
net4.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net4.save_weights('net4.h5')  # each split will start with same weights so can directly compare results

histories4 = []
results_list4 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net4.load_weights('net4.h5')
    history = net4.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories4.append(history)
    
    
    results = net4.evaluate(test_data, test_labels)
    results_list4.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration4 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 32ms/step - loss: 3.3153 - accuracy: 0.1667 - val_loss: 2.6669 - val_accuracy: 0.2222
Epoch 2/25
54/54 [==============================] - 2s 28ms/step - loss: 5.4049 - accuracy: 0.1296 - val_loss: 5.7108 - val_accuracy: 0.2778
Epoch 3/25
54/54 [==============================] - 2s 28ms/step - loss: 5.9114 - accuracy: 0.2407 - val_loss: 2.6081 - val_accuracy: 0.2222
Epoch 4/25
54/54 [==============================] - 2s 28ms/step - loss: 2.2264 - accuracy: 0.2778 - val_loss: 2.7504 - val_accuracy: 0.2222
Epoch 5/25
54/54 [==============================] - 2s 28ms/step - loss: 2.3302 - accuracy: 0.2963 - val_loss: 1.8341 - val_accuracy: 0.3333
Epoch 6/25
54/54 [==============================] - 2s 28ms/step - loss: 1.5407 - accuracy: 0.4074 - val_loss: 1.4990 - val_accuracy: 0.3333
Epoch 7/25
54/54 [==============================] - 1s 28ms/step - loss: 1.3167 - accuracy: 0.3148 - val_los

In [23]:
avg = np.mean(results_list4, axis=0)
std_dev = np.std(results_list4, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration4))

Average loss and accuracy scores were:  [0.51553057 0.73333335]
Standard deviation for loss & accuracy:  [0.03838079 0.08164965]
Duration in seconds:  154


<h2> Expriment5:</h2>

In [28]:
net5 = models.Sequential()
net5.add(layers.Dense((480), input_shape = (480,6)))
net5.add(layers.Flatten())
net5.add(layers.Dense(240,activation='selu'))
net5.add(layers.Dense(240,activation='selu'))
net5.add(layers.Dense(240,activation='selu'))
net5.add(layers.Dense(7, activation = 'softmax'))
net5.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 480, 480)          3360      
_________________________________________________________________
flatten_9 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_42 (Dense)             (None, 240)               55296240  
_________________________________________________________________
dense_43 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_44 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_45 (Dense)             (None, 7)                 1687      
Total params: 55,416,967
T

In [29]:
net5.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net5.save_weights('net5.h5')  # each split will start with same weights so can directly compare results

histories5 = []
results_list5 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net5.load_weights('net5.h5')
    history = net5.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories5.append(history)
    
    
    results = net5.evaluate(test_data, test_labels)
    results_list5.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration5 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 32ms/step - loss: 10.6962 - accuracy: 0.1111 - val_loss: 14.1889 - val_accuracy: 0.0556
Epoch 2/25
54/54 [==============================] - 1s 26ms/step - loss: 17.1072 - accuracy: 0.1667 - val_loss: 21.1174 - val_accuracy: 0.1111
Epoch 3/25
54/54 [==============================] - 1s 27ms/step - loss: 17.3418 - accuracy: 0.1667 - val_loss: 4.3712 - val_accuracy: 0.0556
Epoch 4/25
54/54 [==============================] - 1s 26ms/step - loss: 3.5426 - accuracy: 0.3519 - val_loss: 1.6168 - val_accuracy: 0.3333
Epoch 5/25
54/54 [==============================] - 1s 27ms/step - loss: 3.1897 - accuracy: 0.3889 - val_loss: 2.7532 - val_accuracy: 0.3333
Epoch 6/25
54/54 [==============================] - 1s 27ms/step - loss: 3.3622 - accuracy: 0.5556 - val_loss: 1.9738 - val_accuracy: 0.4444
Epoch 7/25
54/54 [==============================] - 1s 26ms/step - loss: 3.1886 - accuracy: 0.4815 - va

In [30]:
avg = np.mean(results_list5, axis=0)
std_dev = np.std(results_list5, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration5))

Average loss and accuracy scores were:  [0.44806673 0.81666668]
Standard deviation for loss & accuracy:  [0.14883588 0.05527708]
Duration in seconds:  146
